In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import skmob

## Reading the DataFrame

### Data original

In [ ]:
path = '../../../DataTelkomsel/2021Desember/Desember2021.csv'
df = pd.read_csv(path)
df = df.drop({'Unnamed: 0'},axis=1)
df.head()

### Data reverse geocoding 

In [5]:
path2 = '../../../DataTelkomsel/2022Januari/RGjanuari2022.csv'
# df2 = pd.read_csv(path2)
# df2 = df2.drop({'Unnamed: 0'},axis=1)
# df2.head()

## 1. Filter Unnecessary Area
Membatasi area data point pada range yg dilakukan analisis

In [6]:
def filter_area(gps, road, filename):
    road_path = road
    
    tdf = skmob.TrajDataFrame.from_file(gps)
    area_shape = gpd.read_file(road_path)
    
    gdf_gps = GeoDataFrame(tdf, geometry=gpd.points_from_xy(tdf['longitude'], tdf['latitude']))
    filtered_data = gdf_gps[gdf_gps.geometry.within(area_shape.geometry.iloc[0])].copy()
    
    filtered_data['datetime_wib'] = pd.to_datetime(filtered_data['datetime_wib'])
    filtered_data['tanggal'] = filtered_data['datetime_wib'].dt.date
    
    filtered_data.to_csv(filename, index=False)    

    filtered_data.info()

    print("Filtering area for ", filename, "succeed")

In [7]:
gps = path2
road = './Malioboro_around/clipping_boundary.geojson'
filename ='filter1_malar_jan.csv'
filter_area(gps, road, filename)

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2398967 entries, 1112335 to 21676305
Data columns (total 10 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Unnamed: 0      int64         
 1   maid            object        
 2   latitude        float64       
 3   longitude       float64       
 4   datetime_wib    datetime64[ns]
 5   geometry        geometry      
 6   Kelurahan/Desa  object        
 7   Kecamatan       object        
 8   Kabupaten       object        
 9   tanggal         object        
dtypes: datetime64[ns](1), float64(2), geometry(1), int64(1), object(5)
memory usage: 201.3+ MB
Filtering area for  filter1_malar_jan.csv succeed


In [ ]:
data = pd.read_csv(f'./{filename}.csv')
data.head()

In [ ]:
data.nunique()